# AN INVOICE CLASSIFIER

## Description

### This application is used to take in a pdf from a user and classify it according to the sender of the pdf, its category/type of invoice and type/category of goods administered

First of all is to acknowledge the type of formats that can come from the user as an input. The input(invoice) can come as one in two formats (*image or pdf*). The application should be able to take in any of the two formats.

In this context, we are to set the targets locally since there is an assumption that their is a use of a webiste for uploading the target. Without the website, the files will be locally stored in this repository for analysis.

The program will first work with pictures to form the model then it will include PDF's later to scan through.

# IMPLEMENTATION

### Importation of libraries

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import os as os
import sys as sys
import PIL
import PIL.Image


### Reading the csv holding the training data 

In [ ]:
pd.read_csv("files\data.csv")

### Gets total number of images in directory

In [ ]:
import os
import glob

# Define the directory path where your images are located
directory_path = 'files/'  # Replace with the actual directory path

# Define a list of image file extensions (add more if needed)
image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.gif', '*.bmp']

# Initialize a variable to store the count of images
total_image_count = 0

# Loop through each image extension and count the images
for extension in image_extensions:
    pattern = os.path.join(directory_path, extension)
    image_files = glob.glob(pattern)
    total_image_count += len(image_files)

# Print the total number of images
print(f'Total number of images in the directory: {total_image_count}')

### We then save the photos is a tensor

In [ ]:
# For every image in the directory we save it to a tensor
image_tensors = []
for extension in image_extensions:
    pattern = os.path.join(directory_path, extension)
    image_files = glob.glob(pattern)
    for file in image_files:
        image = tf.io.read_file(file)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.convert_image_dtype(image, tf.float32)
        image_tensors.append(image)

desired_size = (256, 256)
resized_image = tf.image.resize(image_tensors, desired_size)

# You can also normalize the image pixels to a specific range (0 to 1):
normalized_image = resized_image / 255.0


### printing some images in the tensors

In [ ]:
# Display the image (optional)
loop = 2
i=0
while i < loop:
    plt.imshow(image_tensors[i])
    plt.axis('off')
    plt.show()
    i += 1


### Now we can start processing the photos. 
The idea is we use neuron network to classify the image according to the recepient of the invoice. It can be companies or individuals.
The csv holds the actual recipient of the invoice and the image is the invoice itself.
The model should be able to analyse the image and pinpoint the recipient of the invoice.
The model should be able to classify the image according to the recipient of the invoice.

In [ ]:
# Train the images in a nuero network
# Define the model 
model = Sequential([
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1)
])
# Compile the model
model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
# Train the model
model.fit(normalized_image, epochs=10)
# Save the model
model.save('model.h5')
# Load the model
model = tf.keras.models.load_model('model.h5')
# Make predictions
predictions = model.predict(normalized_image)
# Print the predictions
print(predictions)
# Print the actual labels
print(labels)
# set the printing of predictions and labels in a loop
loop = 3
i=0
while i < loop:
    print(predictions[i])
    print(labels[i])
    plt.imshow(image_tensors[i])
    plt.axis('off')
    plt.show()
    i += 1


In [34]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential
import matplotlib.pyplot as plt
import os
import glob

# Define the directory path where your images are located
directory_path = 'files/'  # Replace with the actual directory path

# Define a list of image file extensions (add more if needed)
image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.gif', '*.bmp']

# Initialize a variable to store the count of images
total_image_count = 0

# Loop through each image extension and count the images
for extension in image_extensions:
    pattern = os.path.join(directory_path, extension)
    image_files = glob.glob(pattern)
    total_image_count += len(image_files)

# Create lists to store image data and labels
image_data = []  # Store your image data here
labels = []  # Store corresponding labels here

# Load and preprocess images
for extension in image_extensions:
    pattern = os.path.join(directory_path, extension)
    image_files = glob.glob(pattern)
    for img_path in image_files:
        img = plt.imread(img_path)
        img = tf.image.resize(img, (256, 256))  # Resize the image if needed
        img = img / 255.0  # Normalize pixel values to [0, 1]
        image_data.append(img)
        # Assuming you have a way to determine labels for each image, append them to 'labels' list.
        # Example: labels.append(0) for class 0, labels.append(1) for class 1, and so on.

# Convert lists to TensorFlow tensors
image_tensors = tf.convert_to_tensor(image_data)
labels = tf.convert_to_tensor(labels)

# Define the model
model = Sequential([
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1)
])

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train the model
model.fit(image_tensors, labels, epochs=10)

# Save the model
model.save('model.h5')

# Load the model
loaded_model = tf.keras.models.load_model('model.h5')

# Make predictions
predictions = loaded_model.predict(image_tensors)

# Print the predictions
print(predictions)

# Print the actual labels
print(labels)

# Display some images and their predictions (you can adjust 'loop' as needed)
loop = min(3, len(image_data))
for i in range(loop):
    print(predictions[i])
    print(labels[i])
    plt.imshow(image_data[i])
    plt.axis('off')
    plt.show()

ValueError: Data cardinality is ambiguous:
  x sizes: 2
  y sizes: 0
Make sure all arrays contain the same number of samples.

In [ ]:
fulldf= pd.conca